In [48]:
%load_ext autoreload
%autoreload 2

In [163]:
import plotly.graph_objects as go
from plotly.subplots import make_subplots

# Example data for the first choropleth map
# data1 = dict(
#     # type = 'choropleth_mapbox',
#     locations = ['USA', 'CAN', 'MEX'],
#     z = [1, 2, 3],
#     locationmode = 'country names',
#     colorscale = 'Blues',
#     colorbar_title = 'Data 1'
# )

import numpy as np
import pandas as pd
from shapely.geometry import Polygon
import geopandas as gpd
import pyproj
pyproj.datadir.set_data_dir('/Users/jm/Code/mapgpt-mvp/.conda/share/proj')
import matplotlib.pyplot as plt
import os
import sys
from dotenv import load_dotenv
import pandas as pd
import pydeck as pdk
from IPython.display import display
sys.path.append('../src')

from infrastructure.gpt4 import GPT4
from infrastructure.postgres_db import PostgresDB
from application.maps.choropleth_map import ChoroplethMap
import plotly.graph_objects as go

load_dotenv()

db_name = os.environ.get("DB_NAME")
db_user = os.environ.get("DB_USER")
db_password = os.environ.get("DB_PASSWORD")


db = PostgresDB(db_name=db_name, db_user=db_user, db_password=db_password)
comunas = db.run_gpd_query("SELECT * FROM comuna")

# Function to extract coordinates
def get_polygon_coordinates(geom):
    if geom.type == 'Polygon':
        return [(x, y) for x, y in geom.exterior.coords]
    elif geom.type == 'MultiPolygon':
        return [[(x, y) for x, y in polygon.exterior.coords] for polygon in geom.geoms][0]
    else:
        return None

comunas = comunas.to_crs(epsg=4326)
np.random.seed(9)
comunas = comunas.assign(
    lat = comunas.geom.centroid.y,
    lon = comunas.geom.centroid.x,
    coordinates = comunas.geom.apply(lambda geom: [list(coord) for coord in geom.geoms[0].exterior.coords]),
    coordinates2 = comunas.geom.apply(get_polygon_coordinates),
    score=lambda _df: np.random.randint(0, 101, size=len(_df)),
    score1=lambda _df: np.random.randint(0, 101, size=len(_df)),
    score2=lambda _df: np.random.randint(0, 101, size=len(_df)),
    score3=lambda _df: np.random.randint(0, 101, size=len(_df)),
    # geom = comunas.geom.simplify(10, preserve_topology=True)


    
)
# comunas.geom = comunas.geom.simplify(10, preserve_topology=True)
comunas = comunas.to_crs(epsg=3857)
c = comunas.set_index("codigo").head(3)
c.geom = c.geom.simplify(10, preserve_topology=True)
d = comunas.set_index("codigo").tail(3)
d.geom = d.geom.simplify(10, preserve_topology=True)


/Users/jm/Code/mapgpt-mvp/.conda/lib/python3.11/site-packages/geopandas/io/sql.py:185: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df = pd.read_sql(
/Users/jm/Code/mapgpt-mvp/.conda/lib/python3.11/site-packages/geopandas/io/sql.py:473: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  return pd.read_sql(spatial_ref_sys_sql, con)
/var/folders/8j/bf4mjlj94czf88p79v8gb24w0000gp/T/ipykernel_91140/993101657.py:56: UserWarning: Geometry is in a geographic CRS. Results from 'centroid' are likely incorrect. Use 'GeoSeries.to_crs()' to re-project geometries to a projected CRS before this operation.

  lat = comunas.geom.centroid.y,
/var/folders/8j/bf4mjlj94czf88p79v8gb24w0000gp/T/ipykerne

In [164]:
from application.maps.bar_chart_map import bar_chart

    
# scale1 = average_bounding_boxes(c.to_crs(epsg=3857).geom)
r = c.assign(
    # bar_chart=bar_chart(c, ["score1", "score2", "score3"], scale=(1e4, 1e2))
    # scale=scale1, max_value=c[["score1", "score2", "score3"]].max(axis=None)
    bar_chart=bar_chart(c, ["score1", "score2", "score3"])
)
r = r.to_crs(epsg=3857)
concatenated_series = pd.concat([c.geom, r.bar_chart])
concatenated_series.explode().explore()

In [165]:
bar_chart1 = bar_chart(c, ["score1", "score2", "score3"])
bar_chart1.explore()

In [168]:
c

,id,nombre,region_id,geom,lat,lon,coordinates,coordinates2,score,score1,score2,score3
codigo,,,,,,,,,,,,
12202,252,Antártica,12,"POLYGON ((-7992818.789 -4056669.319, -7992731....",-34.372586,-71.671741,"[[-71.80071281536047, -34.20728429832156], [-7...","[(-71.80071281536047, -34.20728429832156), (-7...",92,45,63,77
6102,80,Codegua,6,"POLYGON ((-7831651.813 -4022443.109, -7831494....",-34.057048,-70.547220,"[[-70.35292523789755, -33.95262958618671], [-7...","[(-70.35292523789755, -33.95262958618671), (-7...",54,78,26,85
6103,81,Coinco,6,"POLYGON ((-7892616.357 -4060673.364, -7892621....",-34.282285,-70.971141,"[[-70.90057905544163, -34.23702569687188], [-7...","[(-70.90057905544163, -34.23702569687188), (-7...",56,42,93,76


In [172]:

# b3 = pd.DataFrame(bar_chart1.apply(lambda poly: poly.geoms).to_list(), columns=["score1", "score2", "score3"], index=bar_chart1.index).rename_axis("category", axis=1).unstack()
# b4 = c[["score1", "score2", "score3"]].rename_axis("category", axis=1).unstack()
# gdf = gpd.GeoDataFrame(pd.concat([b4.rename("value"), b3.rename("poly")], axis=1), geometry='poly', crs=c.crs).reset_index()
# gdf.explore()

from application.maps.bar_chart_map import bar_chart_gdf


bb1 = bar_chart_gdf(c, ["score1", "score2", "score3"])
bb1.explore()

In [ ]:
bar_chart1

codigo
12202    MULTIPOLYGON (((-7986088.489 -4088683.391, -79...
6102     MULTIPOLYGON (((-7860910.918 -4046201.846, -78...
6103     MULTIPOLYGON (((-7908099.6 -4076503.098, -7903...
dtype: geometry

In [156]:
pd.DataFrame(bar_chart1.apply(lambda poly: poly.geoms).to_list(), columns=["score1", "score2", "score3"], index=bar_chart1.index)

,score1,score2,score3
codigo,,,
12202,POLYGON ((-7986088.4887401 -4088683.3912085914...,POLYGON ((-7981003.590173433 -4088683.39120859...,POLYGON ((-7975918.691606767 -4088683.39120859...
6102,POLYGON ((-7860910.917515347 -4046201.84627656...,POLYGON ((-7855826.01894868 -4046201.846276568...,POLYGON ((-7850741.120382014 -4046201.84627656...
6103,POLYGON ((-7908099.5997970365 -4076503.0981770...,POLYGON ((-7903014.7012303695 -4076503.0981770...,POLYGON ((-7897929.802663703 -4076503.09817700...


In [151]:
c[["score1", "score2", "score3"]].rename_axis("category", axis=1)

category,score1,score2,score3
codigo,,,
12202,45,63,77
6102,78,26,85
6103,42,93,76


In [147]:
gpd.GeoDataFrame(pd.concat([b4.rename("value"), b3.rename("poly")], axis=1), geometry='poly', crs=c.crs)

value                                               poly
       codigo                                                          
score1 12202      45  POLYGON ((-7986088.489 -4088683.391, -7981003....
       6102       78  POLYGON ((-7860910.918 -4046201.846, -7855826....
       6103       42  POLYGON ((-7908099.6 -4076503.098, -7903014.70...
score2 12202      63  POLYGON ((-7981003.59 -4088683.391, -7975918.6...
       6102       26  POLYGON ((-7855826.019 -4046201.846, -7850741....
       6103       93  POLYGON ((-7903014.701 -4076503.098, -7897929....
score3 12202      77  POLYGON ((-7975918.692 -4088683.391, -7970833....
       6102       85  POLYGON ((-7850741.12 -4046201.846, -7845656.2...
       6103       76  POLYGON ((-7897929.803 -4076503.098, -7892844....

In [140]:
gpd.GeoDataFrame(b4.rename("value"), geometry=b3, crs=c.crs).reset_index()

value                                           geometry
       codigo                                                          
score1 12202      45  POLYGON ((-7986088.489 -4088683.391, -7981003....
       6102       78  POLYGON ((-7860910.918 -4046201.846, -7855826....
       6103       42  POLYGON ((-7908099.6 -4076503.098, -7903014.70...
score2 12202      63  POLYGON ((-7981003.59 -4088683.391, -7975918.6...
       6102       26  POLYGON ((-7855826.019 -4046201.846, -7850741....
       6103       93  POLYGON ((-7903014.701 -4076503.098, -7897929....
score3 12202      77  POLYGON ((-7975918.692 -4088683.391, -7970833....
       6102       85  POLYGON ((-7850741.12 -4046201.846, -7845656.2...
       6103       76  POLYGON ((-7897929.803 -4076503.098, -7892844....

<Projected CRS: EPSG:3857>
Name: WGS 84 / Pseudo-Mercator
Axis Info [cartesian]:
- X[east]: Easting (metre)
- Y[north]: Northing (metre)
Area of Use:
- name: World between 85.06°S and 85.06°N.
- bounds: (-180.0, -85.06, 180.0, 85.06)
Coordinate Operation:
- name: Popular Visualisation Pseudo-Mercator
- method: Popular Visualisation Pseudo Mercator
Datum: World Geodetic System 1984 ensemble
- Ellipsoid: WGS 84
- Prime Meridian: Greenwich

In [127]:
c[["score1", "score2", "score3"]].unstack()

        codigo
score1  12202     45
        6102      78
        6103      42
score2  12202     63
        6102      26
        6103      93
score3  12202     77
        6102      85
        6103      76
dtype: int64

In [130]:
bar_chart1

codigo
12202    MULTIPOLYGON (((-7986088.489 -4088683.391, -79...
6102     MULTIPOLYGON (((-7860910.918 -4046201.846, -78...
6103     MULTIPOLYGON (((-7908099.6 -4076503.098, -7903...
dtype: geometry

In [120]:
pd.DataFrame(bar_chart1.apply(lambda poly: poly.geoms).to_list())

,0,1,2
0,POLYGON ((-7986088.4887401 -4088683.3912085914...,POLYGON ((-7981003.590173433 -4088683.39120859...,POLYGON ((-7975918.691606767 -4088683.39120859...
1,POLYGON ((-7860910.917515347 -4046201.84627656...,POLYGON ((-7855826.01894868 -4046201.846276568...,POLYGON ((-7850741.120382014 -4046201.84627656...
2,POLYGON ((-7908099.5997970365 -4076503.0981770...,POLYGON ((-7903014.7012303695 -4076503.0981770...,POLYGON ((-7897929.802663703 -4076503.09817700...


In [123]:
gdf.index

MultiIndex([('score1', 12202),
            ('score1',  6102),
            ('score1',  6103),
            ('score2', 12202),
            ('score2',  6102),
            ('score2',  6103),
            ('score3', 12202),
            ('score3',  6102),
            ('score3',  6103)],
           names=[None, 'codigo'])

In [111]:
pd.concat([b4.rename("value"), b3.rename("poly")], axis=1)

# Convert pandas DataFrame to GeoDataFrame


# Set the coordinate reference system (CRS) of the GeoDataFrame
gdf.crs = 'EPSG:3857'

# Print the GeoDataFrame
print(gdf)

0                                                  1
       codigo                                                       
score1 12202   45  POLYGON ((-7986088.4887401 -4088683.3912085914...
       6102    78  POLYGON ((-7860910.917515347 -4046201.84627656...
       6103    42  POLYGON ((-7908099.5997970365 -4076503.0981770...
score2 12202   63  POLYGON ((-7981003.590173433 -4088683.39120859...
       6102    26  POLYGON ((-7855826.01894868 -4046201.846276568...
       6103    93  POLYGON ((-7903014.7012303695 -4076503.0981770...
score3 12202   77  POLYGON ((-7975918.691606767 -4088683.39120859...
       6102    85  POLYGON ((-7850741.120382014 -4046201.84627656...
       6103    76  POLYGON ((-7897929.802663703 -4076503.09817700...

        codigo
score1  12202     POLYGON ((-7986088.4887401 -4088683.3912085914...
        6102      POLYGON ((-7860910.917515347 -4046201.84627656...
        6103      POLYGON ((-7908099.5997970365 -4076503.0981770...
score2  12202     POLYGON ((-7981003.590173433 -4088683.39120859...
        6102      POLYGON ((-7855826.01894868 -4046201.846276568...
        6103      POLYGON ((-7903014.7012303695 -4076503.0981770...
score3  12202     POLYGON ((-7975918.691606767 -4088683.39120859...
        6102      POLYGON ((-7850741.120382014 -4046201.84627656...
        6103      POLYGON ((-7897929.802663703 -4076503.09817700...
dtype: object

In [84]:
bar_chart

<function application.maps.bar_chart_map.bar_chart(df: geopandas.geodataframe.GeoDataFrame, values_cols: list[str], **kwargs) -> geopandas.geoseries.GeoSeries>

In [80]:
str(c.crs)

'EPSG:3857'

In [181]:
c

,id,nombre,region_id,geom,lat,lon,coordinates,coordinates2,score,score1,score2,score3
codigo,,,,,,,,,,,,
12202,252,Antártica,12,"POLYGON ((-7992818.789 -4056669.319, -7992731....",-34.372586,-71.671741,"[[-71.80071281536047, -34.20728429832156], [-7...","[(-71.80071281536047, -34.20728429832156), (-7...",92,45,63,77
6102,80,Codegua,6,"POLYGON ((-7831651.813 -4022443.109, -7831494....",-34.057048,-70.547220,"[[-70.35292523789755, -33.95262958618671], [-7...","[(-70.35292523789755, -33.95262958618671), (-7...",54,78,26,85
6103,81,Coinco,6,"POLYGON ((-7892616.357 -4060673.364, -7892621....",-34.282285,-70.971141,"[[-70.90057905544163, -34.23702569687188], [-7...","[(-70.90057905544163, -34.23702569687188), (-7...",56,42,93,76


In [191]:
from folium.features import GeoJsonTooltip

m = folium.Map(location=[-33.45, -70.66], zoom_start=10)

bb2 = bar_chart_gdf(c, ["score1", "score2", "score3"])

colors = {
    "score1": "blue",
    "score2": "green",
    "score3": "red"
}

folium.GeoJson(c, tooltip=GeoJsonTooltip(fields=["nombre"], aliases=["Nombre"], localize=True)).add_to(m)

folium.GeoJson(bb2, style_function=lambda feature: {'fillColor': colors.get(feature['properties']['category']), 'fillOpacity':0.8}, tooltip=GeoJsonTooltip(fields=["category", "value"], aliases=["Candidato", "Valor"], localize=True)).add_to(m)

m

m 

In [190]:
bb2

,category,codigo,value,poly
0,score1,12202,45,"POLYGON ((-7986088.489 -4088683.391, -7981003...."
1,score1,6102,78,"POLYGON ((-7860910.918 -4046201.846, -7855826...."
2,score1,6103,42,"POLYGON ((-7908099.6 -4076503.098, -7903014.70..."
3,score2,12202,63,"POLYGON ((-7981003.59 -4088683.391, -7975918.6..."
4,score2,6102,26,"POLYGON ((-7855826.019 -4046201.846, -7850741...."
5,score2,6103,93,"POLYGON ((-7903014.701 -4076503.098, -7897929...."
6,score3,12202,77,"POLYGON ((-7975918.692 -4088683.391, -7970833...."
7,score3,6102,85,"POLYGON ((-7850741.12 -4046201.846, -7845656.2..."
8,score3,6103,76,"POLYGON ((-7897929.803 -4076503.098, -7892844...."


In [41]:
c.geom

codigo
12202    POLYGON ((-7992818.789 -4056669.319, -7992731....
6102     POLYGON ((-7831651.813 -4022443.109, -7831494....
6103     POLYGON ((-7892616.357 -4060673.364, -7892621....
Name: geom, dtype: geometry

In [39]:
scale1

(30509.391399999615, 38927.0913666673)

In [4]:
m.save('map.html')

In [5]:
r.bar_chart.explode()

codigo
12202    POLYGON ((-7993461.141 -4078951.618, -7983461....
12202    POLYGON ((-7983461.141 -4078951.618, -7973461....
12202    POLYGON ((-7973461.141 -4078951.618, -7963461....
6102     POLYGON ((-7868283.57 -4036470.073, -7858283.5...
6102     POLYGON ((-7858283.57 -4036470.073, -7848283.5...
6102     POLYGON ((-7848283.57 -4036470.073, -7838283.5...
6103     POLYGON ((-7915472.252 -4066771.325, -7905472....
6103     POLYGON ((-7905472.252 -4066771.325, -7895472....
6103     POLYGON ((-7895472.252 -4066771.325, -7885472....
Name: bar_chart, dtype: geometry

In [6]:


concatenated_series.explode().explore()

In [7]:
concatenated_series

codigo
12202    POLYGON ((-7992818.789 -4056669.319, -7992731....
6102     POLYGON ((-7831651.813 -4022443.109, -7831494....
6103     POLYGON ((-7892616.357 -4060673.364, -7892621....
12202    MULTIPOLYGON (((-7993461.141 -4078951.618, -79...
6102     MULTIPOLYGON (((-7868283.57 -4036470.073, -785...
6103     MULTIPOLYGON (((-7915472.252 -4066771.325, -79...
dtype: geometry

In [8]:
c.explore()

In [9]:
concatenated_df = pd.concat([c, d])

In [25]:
from functools import reduce


def agg_boxes(b1: tuple, b2: tuple):
    return tuple([(b1[i] + b2[i]/2) for i in range(len(b1))])
apply(lambda x: x.bounds)

reduce(agg_boxes, comunas.geom.bounds)

TypeError: unsupported operand type(s) for /: 'str' and 'int'

(65772.7873048922, 70314.0962282205)

In [ ]:
minx, miny, maxx, maxy = polygon.bounds
width = maxx - minx

height = maxy - miny

In [27]:
from shapely.geometry import Polygon

# comunas.geom.bounds.mean()
miny, minx, maxy, maxx = comunas.geom.bounds.mean()
polygon = Polygon([(minx, miny), (maxx, miny), (maxx, maxy), (minx, maxy), (minx, miny)])

minx   -8.021711e+06
miny   -4.306925e+06
maxx   -7.951397e+06
maxy   -4.241152e+06
dtype: float64

In [29]:
miny, minx, maxy, maxx = comunas.geom.bounds.mean()
maxx

-4241151.845687637

In [ ]:
comunas.geom.apply(lambda x: x.bounds)

comunas.geom.apply(lambda x: x.bounds).agg


def average_bounding_boxes(s: gpd.GeoSeries):
    return s.apply(lambda x: x.bounds).mean(axis=0)


TypeError: complex() first argument must be a string or a number, not 'tuple'

In [18]:
comunas.geom.apply(lambda x: x.bounds)

comunas.geom.apply(lambda x: x.bounds).agg


def average_bounding_boxes(s: gpd.GeoSeries):
    return s.apply(lambda x: x.bounds).mean(axis=0)


TypeError: complex() first argument must be a string or a number, not 'tuple'

In [11]:
from functools import reduce


average_bounding_boxes(comunas.geom)

def agg_

reduce(lambda x, y: x + y, comunas.geom.apply(lambda x: x.bounds))

TypeError: complex() first argument must be a string or a number, not 'tuple'